In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers
from keras.api.callbacks import ModelCheckpoint

from keras.api import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


2025-05-02 02:04:14.317393: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 02:04:14.538960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746122654.639953   15459 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746122654.667513   15459 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746122654.859361   15459 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Open the HDF5 file in read mode
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292) float32
(5000, 7) int8


In [3]:
label_counts = np.sum(mid_level_label_data, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


Melodiousness: 1340
Articulation: 907
Rhythmic Complexity: 295
Rhythmic Stability: 1540
Dissonance: 440
Tonal Stability: 2375
Modality: 813


In [4]:
# Open the HDF5 file in read mode
with h5py.File("Data/Emotify_Annotated_Data.h5", "r") as hf:
    emotify_train_data = hf["train"][:]  # Load the train dataset
    emotify_label_data = hf["label"][:]  # Load the label dataset

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

(400, 256, 1292) float32
(400, 7) int8


In [5]:
label_counts = np.sum(emotify_label_data, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


Melodiousness: 103
Articulation: 88
Rhythmic Complexity: 19
Rhythmic Stability: 81
Dissonance: 27
Tonal Stability: 208
Modality: 67


In [6]:
combined_train = np.concatenate([mid_level_train_data, emotify_train_data], axis=0)
combined_label = np.concatenate([mid_level_label_data, emotify_label_data], axis=0)

In [7]:
# Shuffle
permutation = np.random.permutation(combined_train.shape[0]) # Randomly permute the indices using "5000 + 400"
shuffled_train = combined_train[permutation]
shuffled_label = combined_label[permutation]

In [8]:
# Print final shapes to verify
print("Shuffled Train Shape:", shuffled_train.shape)
print("Shuffled Label Shape:", shuffled_label.shape)

Shuffled Train Shape: (5400, 256, 1292)
Shuffled Label Shape: (5400, 7)


In [9]:
label_counts = np.sum(shuffled_label, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


Melodiousness: 1443
Articulation: 995
Rhythmic Complexity: 314
Rhythmic Stability: 1621
Dissonance: 467
Tonal Stability: 2583
Modality: 880


In [10]:
shuffled_train = shuffled_train[..., np.newaxis]  # Add a new axis to the last dimension

In [11]:

print("Shuffled Train Shape:", shuffled_train.shape)

Shuffled Train Shape: (5400, 256, 1292, 1)


In [12]:
shuffled_train.dtype

dtype('float32')

In [13]:
# mid_level_train_data = mid_level_train_data[..., np.newaxis]

# print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
# print(mid_level_label_data.shape, mid_level_label_data.dtype)

In [14]:
# emotify_train_data = emotify_train_data[..., np.newaxis]

# print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
# print(emotify_label_data.shape, emotify_label_data.dtype)

In [15]:
# mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
# emotify_input_shape = emotify_train_data.shape[1], emotify_train_data.shape[2], emotify_train_data.shape[3]
# print(mid_level_input_shape)
# print(mid_level_input_shape, emotify_input_shape)

In [16]:
# with tensorflow.device('/GPU:0'):
#     input = Input(shape=(256, 1292, 1))
#     x =  Conv2D(64, (5, 5), strides=2, activation="relu", padding="valid")(input)
#     x =  BatchNormalization()(x)   
#     # 2nd Layer
#     x =  Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 3rd Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 4th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 5th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 6th Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 7th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 8th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 9th Layer
#     x =  Conv2D(384, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 10th Layer
#     x =  Conv2D(512, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 11th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x) 
#     # 12th Layer
#     # x = tfa.layers.AdaptiveAveragePooling2D(x)
#     x =  GlobalAveragePooling2D(keepdims=True)(x)
#     x =  Flatten()(x)
#     x = keras.layers.Dense(256)(x)
#     A2Mid2E_branch = keras.layers.Dense(7, activation="sigmoid")(x)
#     model = Model(inputs=input, outputs=A2Mid2E_branch, name="Mid-Level_Features")
#     model.summary()

In [17]:
model = load_model("model_epoch_52_acc_0.82_loss_0.02_valacc_0.91_valloss_0.02.keras")
# model.summary() 

I0000 00:00:1746122856.286307   15459 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1746122856.291622   15459 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath="checkpoint_epoch-{epoch:02d}_acc-{accuracy:.4f}_loss-{loss:.4f}.keras",
    monitor='accuracy',          # Monitor training accuracy
    mode='max',                   # Save when training accuracy improves
    save_best_only=True,          # Save only the best model
    save_weights_only=False,      # Save the entire model
    verbose=1
)
model.compile(optimizer=Adam(learning_rate=0.0005),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )
model.fit(mid_level_train_data,
          mid_level_label_data,
          batch_size=8,
          epochs=54,
          initial_epoch=52,
          callbacks=[checkpoint_callback],
          shuffle=True, verbose=1)